In [1]:
# Install required libraries
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.1 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset
import torch
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from tqdm import tqdm
import random

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load the IMDB dataset
imdb = load_dataset("imdb")
train_dataset = imdb['train'].shuffle(seed=42)
test_dataset = imdb['test'].shuffle(seed=42)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# For hyperparameter tuning, create a validation split
val_split_ratio = 0.1
val_size = int(len(train_dataset) * val_split_ratio)

val_dataset = train_dataset.select([i for i in list(range(val_size))])
train_dataset = train_dataset.select([i for i in list(range(val_size, len(train_dataset)))])

print(f"Train dataset size after split: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")


Train dataset size: 25000
Test dataset size: 25000
Train dataset size after split: 22500
Validation dataset size: 2500


In [11]:
# Load LLaMA 3.2 tokenizer
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Update tokenizer with the pad token
num_added_tokens = tokenizer.add_special_tokens({'pad_token': tokenizer.pad_token})
print(f"Added {num_added_tokens} tokens to the tokenizer")

# Load LLaMA for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    pad_token_id=tokenizer.pad_token_id
)

# Resize embeddings to account for added tokens
model.resize_token_embeddings(len(tokenizer))

# Create a small subset of examples to use as demonstration shots
def get_shot_examples(dataset, num_shots=5):
    # Get equal positive and negative examples
    pos_examples = [ex for ex in dataset if ex['label'] == 1][:num_shots//2 + num_shots%2]
    neg_examples = [ex for ex in dataset if ex['label'] == 0][:num_shots//2]

    # Shuffle to avoid bias from ordering
    examples = pos_examples + neg_examples
    random.shuffle(examples)

    return examples

shot_examples = get_shot_examples(train_dataset)
print(f"Selected {len(shot_examples)} shot examples")

# Format the input using the 5 example shots plus the target example
def format_5shot_input(example):
    # Start with a general instruction
    formatted_text = "Classify the sentiment of movie reviews as positive or negative.\n\n"

    # Add the 5 examples with their labels
    for i, shot in enumerate(shot_examples):
        sentiment = "positive" if shot["label"] == 1 else "negative"
        # Truncate long reviews to keep sequence length manageable
        truncated_text = shot["text"][:150] + "..." if len(shot["text"]) > 150 else shot["text"]
        formatted_text += f"Example {i+1}:\nReview: {truncated_text}\nSentiment: {sentiment}\n\n"

    # Add the target example to classify (without label)
    formatted_text += f"Example {len(shot_examples)+1}:\nReview: {example['text']}\nSentiment: "

    return formatted_text

# Define tokenization function
def tokenize_function(examples):
    # Format each example with the 5-shot context
    formatted_texts = [format_5shot_input({"text": text}) for text in examples["text"]]

    # Tokenize with appropriate padding and truncation
    return tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=512  # Increased to accommodate 5-shot examples
    )


Added 0 tokens to the tokenizer


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Selected 5 shot examples


In [12]:
# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Format datasets
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [13]:
# Define evaluation metrics (same as original)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Load metrics
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")

    # Calculate metrics
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1_score = f1.compute(predictions=predictions, references=labels, average="binary")["f1"]
    precision_score = precision.compute(predictions=predictions, references=labels, average="binary")["precision"]
    recall_score = recall.compute(predictions=predictions, references=labels, average="binary")["recall"]

    return {
        "accuracy": accuracy_score,
        "f1": f1_score,
        "precision": precision_score,
        "recall": recall_score
    }

In [14]:
# Define training arguments (adjusted for longer sequences)
training_args = TrainingArguments(
    output_dir="./llama_3.2_5shot_classification_imdb_results",
    learning_rate=2e-5,  # Lower learning rate for few-shot
    per_device_train_batch_size=2,  # Reduced batch size due to longer sequences
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./llama_3.2_5shot_classification_imdb_logs",
    logging_steps=200,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="tensorboard",
    gradient_accumulation_steps=8,  # Increased accumulation to handle smaller batches
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# Train the model
print("Starting training...")
train_results = trainer.train()
print(train_results)

# Evaluate on validation set
print("Evaluating on validation set...")
val_results = trainer.evaluate(eval_dataset=tokenized_val)
print(val_results)

# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(eval_dataset=tokenized_test)
print(test_results)

# Save the final model
model_path = "./llama_3.2_5shot_classification_imdb_sentiment"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.237400,0.284813,0.906400,0.912030,0.864576,0.964996
2,0.101100,0.354807,0.929600,0.931677,0.909780,0.954654
3,0.039700,0.450251,0.926000,0.928433,0.903614,0.954654
4,0.000000,0.692238,0.928400,0.928257,0.935380,0.921241


TrainOutput(global_step=7030, training_loss=0.09306093011445529, metrics={'train_runtime': 23861.6018, 'train_samples_per_second': 4.715, 'train_steps_per_second': 0.295, 'total_flos': 3.361174246989496e+17, 'train_loss': 0.09306093011445529, 'epoch': 4.9969777777777775})
Evaluating on validation set...


{'eval_loss': 0.35480743646621704, 'eval_accuracy': 0.9296, 'eval_f1': 0.9316770186335404, 'eval_precision': 0.9097801364670205, 'eval_recall': 0.954653937947494, 'eval_runtime': 172.987, 'eval_samples_per_second': 14.452, 'eval_steps_per_second': 7.226, 'epoch': 4.9969777777777775}
Evaluating on test set...
{'eval_loss': 0.36330848932266235, 'eval_accuracy': 0.92512, 'eval_f1': 0.9269947741985805, 'eval_precision': 0.904352457768985, 'eval_recall': 0.9508, 'eval_runtime': 1728.181, 'eval_samples_per_second': 14.466, 'eval_steps_per_second': 7.233, 'epoch': 4.9969777777777775}
Model saved to ./llama_3.2_5shot_classification_imdb_sentiment


In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./llama_3.2_5shot_classification_imdb_results",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./llama_3.2_5shot_classification_imdb_logs",
    logging_steps=200,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="tensorboard",
    gradient_accumulation_steps=8,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
print("Starting training...")
train_results = trainer.train()
print(train_results)

# Evaluate on validation set
print("Evaluating on validation set...")
val_results = trainer.evaluate(eval_dataset=tokenized_val)
print(val_results)

# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(eval_dataset=tokenized_test)
print(test_results)

# Save the final model
model_path = "./llama_3.2_5shot_classification_imdb_sentiment"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.084000,0.630412,0.898800,0.892111,0.961397,0.832140
2,0.012200,0.676632,0.924000,0.922449,0.947192,0.898966


In [ ]:
# Function to test a few examples
def test_example(text, model, tokenizer):
    formatted_input = format_5shot_input({"text": text})
    inputs = tokenizer(formatted_input, return_tensors="pt", padding=True)

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    sentiment = "positive" if prediction == 1 else "negative"

    print(f"Review (truncated): {text[:100]}...")
    print(f"Predicted sentiment: {sentiment}")
    print(f"Confidence: {torch.softmax(logits, dim=-1)[0][prediction].item():.4f}")
    print("---")

# Test a few examples from the test set
print("\nTesting a few examples:")
for i in range(5):
    test_example(test_dataset[i]["text"], model, tokenizer)